In [1]:
import argparse
import base64
import json

import numpy as np
import socketio
import eventlet
import eventlet.wsgi
import time
from PIL import Image
from PIL import ImageOps
from flask import Flask, render_template
from io import BytesIO

import cv2

from keras.models import load_model
from keras.models import model_from_json
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array

Using TensorFlow backend.


In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

scale_factor = 4

def normalize_channel(image_data):
    x_min = np.min(image_data)
    x_max = np.max(image_data)
    # Non extreme values are useful to avoid saturation for some act funcs
    a     = -0.5
    b     =  0.5
    return a + np.divide( (image_data - x_min ) * ( b - a), x_max - x_min) 

def preprocess_image(img):
    return normalize_channel(img) 



In [3]:
sio = socketio.Server()
app = Flask(__name__)
model = load_model('weights-improvement-04-0.0268.hdf5')
prev_image_array = None

In [ ]:
@sio.on('telemetry')
def telemetry(sid, data):
    # The current steering angle of the car
    steering_angle = data["steering_angle"]
    # The current throttle of the car
    throttle = data["throttle"]
    # The current speed of the car
    speed = data["speed"]
    # The current image from the center camera of the car
    imgString = data["image"]
    image = Image.open(BytesIO(base64.b64decode(imgString)))
    image_array = np.asarray(image)[60:140,:,:]
    image_center = cv2.resize(image_array, (0,0), fx=1/scale_factor, fy=1/scale_factor, interpolation=cv2.INTER_AREA)
    image_center = preprocess_image(image_center)
    transformed_image_array = image_center.reshape(( 1, image_center.shape[0], image_center.shape[1], 3))
    
    # This model currently assumes that the features of the model are just the images. Feel free to change this.
    steering_angle = float(model.predict(transformed_image_array, batch_size=1))
    # The driving model currently just outputs a constant throttle. Feel free to edit this.
    throttle = 0.2
    print(steering_angle, throttle)
    send_control(steering_angle, throttle)


@sio.on('connect')
def connect(sid, environ):
    print("connect ", sid)
    send_control(0, 0)


def send_control(steering_angle, throttle):
    sio.emit("steer", data={
    'steering_angle': steering_angle.__str__(),
    'throttle': throttle.__str__()
    }, skip_sid=True)

In [ ]:
#parser = argparse.ArgumentParser(description='Remote Driving')
#parser.add_argument('model', type=str,
#help='Path to model definition json. Model weights should be on the same path.')
#args = parser.parse_args()
#with open(args.model, 'r') as jfile:
#    model = model_from_json(json.load(jfile))
#
#model.compile("adam", "mse")
#weights_file = args.model.replace('json', 'h5')
#model.load_weights(weights_file)

# wrap Flask application with engineio's middleware
app = socketio.Middleware(sio, app)

# deploy as an eventlet WSGI server
eventlet.wsgi.server(eventlet.listen(('', 4567)), app)

(35208) wsgi starting up on http://0.0.0.0:4567
(35208) accepted ('127.0.0.1', 63829)


connect  eeca7cf87c6d4c2684270b3c593018b5
-0.10437368601560593 0.2
-0.10437368601560593 0.2
-0.10437368601560593 0.2
-0.1003255546092987 0.2
-0.1003255546092987 0.2
-0.1003255546092987 0.2
-0.10038194805383682 0.2
-0.10038194805383682 0.2
-0.10038194805383682 0.2
-0.07246103882789612 0.2
-0.07246103882789612 0.2
-0.07246103882789612 0.2
-0.05321970582008362 0.2
-0.05321970582008362 0.2
-0.05321970582008362 0.2
-0.04745449125766754 0.2
-0.04745449125766754 0.2
-0.04745449125766754 0.2
-0.12098527699708939 0.2
-0.12098527699708939 0.2
-0.12098527699708939 0.2
-0.05022187530994415 0.2
-0.05022187530994415 0.2
-0.05022187530994415 0.2
-0.03977380320429802 0.2
-0.03977380320429802 0.2
-0.03977380320429802 0.2
-0.03648730367422104 0.2
-0.03648730367422104 0.2
-0.03648730367422104 0.2
-0.06552078574895859 0.2
-0.06552078574895859 0.2
-0.06552078574895859 0.2
-0.07043977826833725 0.2
-0.07043977826833725 0.2
-0.07043977826833725 0.2
-0.07245173305273056 0.2
-0.07245173305273056 0.2
-0.07245173